# **Tiempo estimado de venta**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "puebla" in nombre

    if nombre.startswith("df_") and "puebla" in nombre    
    and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_puebla',
 'df_feb_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [5]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Antiguedad':'antiguedad'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [6]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','antiguedad']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

## Meses de antigüedad

In [8]:
def categorizar_antiguedad_2023(valor):
    if valor == 0:
        return '7'
    elif valor == 1:
        return '8'
    elif valor == 2:
        return '9'
    elif valor == 3:
        return '10'
    elif valor == 4:
        return '11'
    elif valor == 5:
        return '12'
    elif 6 <= valor <= 15:
        return '13-19'
    elif 16 <= valor <= 30:
        return '20-34'
    else:
        return '>=35'

for df in dataframes_list:
    df.loc[:, 'antiguedad_categoria'] = df['antiguedad'].apply(categorizar_antiguedad_2023)    
    #df['antiguedad_categoria'] = df['antiguedad'].apply(categorizar_antiguedad_2023)
    print(df)

      id categoria  antiguedad antiguedad_categoria
0      1        C1           0                    7
1      2         S           0                    7
2      3        C1           0                    7
3      4         S           2                    9
4      5        D1           0                    7
..   ...       ...         ...                  ...
181  182         S          20                20-34
182  183         S           0                    7
183  184         S           1                    8
184  185         S          30                20-34
185  186         S          30                20-34

[186 rows x 4 columns]
        id categoria  antiguedad antiguedad_categoria
0        1        E1           0                    7
1        2        E1           0                    7
2        3        E1           0                    7
3        4        E1           0                    7
4        5        E1           0                    7
...    ...       ...        

Conteo único de coincidencias con la antigüedad

In [10]:
# Orden personalizado
orden_personalizado = ['7', '8', '9', '10', '11', '12', '13-19','20-34','>=35']

tabla_suma = None
# Iterar sobre cada DataFrame en la lista
for df in dataframes_list:
    # Crear una nueva columna con la primera letra de la categoría
    #df['segmento'] = df['categoria'].str[0]
    df.loc[:, 'segmento'] = df['categoria'].str[0]
    #df.drop(columns=['categoria'], inplace=True)        
    # Agrupar y contar los datos
    tabla_actual = df.groupby(['segmento', 'antiguedad_categoria'],observed=False).size().unstack(fill_value=0)
    
    # Sumar la tabla actual al acumulador (tabla_suma)
    if tabla_suma is None:
        tabla_suma = tabla_actual
    else:
        tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)

# Reordenar las columnas según el orden personalizado
tabla_suma = tabla_suma[orden_personalizado]

# Imprimir la tabla acumulativa
print(tabla_suma)

antiguedad_categoria       7      8     9     10    11    12  13-19  20-34  \
segmento                                                                     
A                     1820.0  109.0  62.0   87.0  29.0  44.0  132.0   95.0   
B                     1950.0  110.0  70.0  122.0  40.0  41.0  127.0   95.0   
C                     2027.0   71.0  53.0   64.0  21.0  42.0  118.0   59.0   
D                     2081.0   61.0  46.0   34.0  10.0  17.0   87.0   50.0   
E                     2885.0    8.0  16.0   11.0   2.0   8.0   42.0   30.0   
L                      566.0    9.0   0.0    2.0   4.0   4.0   33.0   25.0   
S                     2980.0  160.0  48.0   86.0  73.0  47.0  198.0  215.0   

antiguedad_categoria   >=35  
segmento                     
A                      18.0  
B                      39.0  
C                      16.0  
D                       7.0  
E                       7.0  
L                       7.0  
S                     108.0  


In [11]:
import os
import plotly.graph_objects as go
import plotly.io as pio

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']
orden_personalizado = ['7', '8', '9', '10', '11', '12', '13-19', '20-34', '>=35']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    for df in dataframes_list:
        #df['segmento'] = df['categoria'].str[0]
        df.loc[:, 'segmento'] = df['categoria'].str[0]
        tabla_actual = df.groupby(['segmento', 'antiguedad_categoria'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    tabla_suma = tabla_suma[orden_personalizado]
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Para guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_tiempoventa', carpeta='assets/graficas')
    
    return fig

def graficar_tabla(tabla):
    segmentos = tabla.index  
    tiempo = tabla.columns
    
    fig = go.Figure()
        
    for i, tiempo_categoria in enumerate(tiempo):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=segmentos,
            y=tabla[tiempo_categoria],
            name=tiempo_categoria,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="Meses de antigüedad",
        yaxis=dict(
            title="Número de casas",
            gridcolor='#dddcda',
        ),
        legend_title="Antigüedad",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        barmode='group',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',
    )
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Supongamos que tienes una lista de DataFrames llamada dataframes_list
fig = sumar_tablas(dataframes_list)


Tabla general de frecuencia:
antiguedad_categoria       7      8     9     10    11    12  13-19  20-34  \
segmento                                                                     
A                     1820.0  109.0  62.0   87.0  29.0  44.0  132.0   95.0   
B                     1950.0  110.0  70.0  122.0  40.0  41.0  127.0   95.0   
C                     2027.0   71.0  53.0   64.0  21.0  42.0  118.0   59.0   
D                     2081.0   61.0  46.0   34.0  10.0  17.0   87.0   50.0   
E                     2885.0    8.0  16.0   11.0   2.0   8.0   42.0   30.0   
L                      566.0    9.0   0.0    2.0   4.0   4.0   33.0   25.0   
S                     2980.0  160.0  48.0   86.0  73.0  47.0  198.0  215.0   

antiguedad_categoria   >=35  
segmento                     
A                      18.0  
B                      39.0  
C                      16.0  
D                       7.0  
E                       7.0  
L                       7.0  
S                     108.0 

## Totales

In [12]:
# Orden personalizado
orden_personalizado = ['7', '8', '9', '10', '11', '12', '13-19','20-34','>=35']

tabla_suma = None
# Iterar sobre cada DataFrame en la lista
for df in dataframes_list:
    # Contar los registros por cada categoría de antigüedad
    tabla_actual = df['antiguedad_categoria'].value_counts().sort_index()    
    
    # Sumar la tabla actual al acumulador (tabla_suma)
    if tabla_suma is None:
        tabla_suma = tabla_actual
    else:
        tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)

# Reordenar las columnas según el orden personalizado
tabla_suma = tabla_suma[orden_personalizado]

# Crear un DataFrame a partir de la tabla acumulativa
df_acumulado = pd.DataFrame(tabla_suma)
df_acumulado.columns = ['count']
df_acumulado['antiguedad_categoria'] = df_acumulado.index

# Reiniciar el índice del DataFrame
df_acumulado.reset_index(drop=True, inplace=True)
df_acumulado

,count,antiguedad_categoria
0,14309.0,7
1,528.0,8
2,295.0,9
3,406.0,10
4,179.0,11
5,203.0,12
6,737.0,13-19
7,569.0,20-34
8,202.0,>=35


In [13]:
fig = go.Figure()
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']
count = df_acumulado['count']

fig.add_trace(go.Bar(
    x=df_acumulado['antiguedad_categoria'], 
    y=df_acumulado['count'], 
    marker_color=colores,  # Especifica los colores de las barras
    text=count,  # Texto que se mostrará en las barras 
    textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
    #texttemplate='%{text:.1f}%',  # Formato del texto (porcentaje con un decimal)
))
# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Meses de antigüedad',  # Título de la gráfica
   xaxis=dict(title='Meses de antigüedad'),  # Título del eje x
    yaxis=dict(
        title='Número de casas', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempoventa_total', carpeta='assets/graficas')

fig.show()